In [12]:
import os
import random
import yaml
import subprocess
import platform
from collections import defaultdict

# Define dataset directory
DIR_PATH = "slakh2100_flac_redux/test"

In [16]:
# Load metadata.yaml from a track folder
def load_metadata(track_path):
    yaml_path = os.path.join(track_path, "metadata.yaml")
    if not os.path.exists(yaml_path):
        return None
    with open(yaml_path, "r") as f:
        return yaml.safe_load(f)

# Iterate through the entire dataset and collect unique plugin names for a given class
def get_unique_plugins_by_class(inst_class):
    unique_plugins = set()
    
    track_folders = [f for f in os.listdir(DIR_PATH) if f.startswith("Track")]
    if not track_folders:
        print("No tracks found.")
        return
    
    for track in track_folders:
        track_path = os.path.join(DIR_PATH, track)
        metadata = load_metadata(track_path)
        
        if not metadata or "stems" not in metadata:
            continue
        
        for stem_data in metadata["stems"].values():
            # Cast program_num to integer before comparison
            if stem_data.get("inst_class") == inst_class:
                unique_plugins.add(stem_data.get("plugin_name", "Unknown"))
    
    print(f"[program_num: {inst_class}]")
    for plugin in sorted(unique_plugins):
        print(plugin)

# Example usage
get_unique_plugins_by_class("Brass")

[program_num: Brass]
None
bass_trombone.nkm
brass_quartet_essential.nkm
flugelhorn.nkm
horn_1_essential.nkm
horn_2_essential.nkm
mute_trumpet.nkm
muted_trumpet.nkm
session_horns_pro_keyswitch_60s_horns.nkm
session_horns_pro_keyswitch_generic_section.nkm
tenor_trombone.nkm
trombone.nkm
trombone_section.nkm
trumpet.nkm
trumpet_2.nkm
tuba.nkm


In [18]:
# List of target plugin names
piano_plugins = {
    "None",
    "bass_trombone.nkm",
    "brass_quartet_essential.nkm",
    "flugelhorn.nkm",
    "horn_1_essential.nkm",
    "horn_2_essential.nkm",
    "mute_trumpet.nkm",
    "muted_trumpet.nkm",
    "session_horns_pro_keyswitch_60s_horns.nkm",
    "session_horns_pro_keyswitch_generic_section.nkm",
    "tenor_trombone.nkm",
    "trombone.nkm",
    "trombone_section.nkm",
    "trumpet.nkm",
    "trumpet_2.nkm",
    "tuba.nkm"
}

# Dictionary to store counts
plugin_counts = defaultdict(int)
total_piano_stems = 0

# Iterate through track directories
for track in os.listdir(DIR_PATH):
    track_path = os.path.join(DIR_PATH, track)
    metadata_path = os.path.join(track_path, "metadata.yaml")

    if os.path.isdir(track_path) and os.path.isfile(metadata_path):
        with open(metadata_path, "r") as file:
            try:
                metadata = yaml.safe_load(file)
                stems = metadata.get("stems", {})

                for stem_data in stems.values():
                    if stem_data.get("inst_class") == "Brass":
                        total_piano_stems += 1
                        plugin_name = stem_data.get("plugin_name")
                        if plugin_name in piano_plugins:
                            plugin_counts[plugin_name] += 1

            except yaml.YAMLError as e:
                print(f"Error parsing {metadata_path}: {e}")

# Print results
print(f"Total Brass Stems: {total_piano_stems}\n")
for plugin, count in sorted(plugin_counts.items(), key=lambda x: -x[1]):
    percentage = (count / total_piano_stems) * 100 if total_piano_stems > 0 else 0
    print(f"{plugin}: {count} ({percentage:.2f}%)")


Total Brass Stems: 71

brass_quartet_essential.nkm: 10 (14.08%)
None: 10 (14.08%)
session_horns_pro_keyswitch_60s_horns.nkm: 9 (12.68%)
session_horns_pro_keyswitch_generic_section.nkm: 8 (11.27%)
horn_2_essential.nkm: 5 (7.04%)
trombone_section.nkm: 5 (7.04%)
mute_trumpet.nkm: 4 (5.63%)
trumpet.nkm: 3 (4.23%)
tuba.nkm: 3 (4.23%)
horn_1_essential.nkm: 3 (4.23%)
tenor_trombone.nkm: 2 (2.82%)
bass_trombone.nkm: 2 (2.82%)
trombone.nkm: 2 (2.82%)
muted_trumpet.nkm: 2 (2.82%)
flugelhorn.nkm: 2 (2.82%)
trumpet_2.nkm: 1 (1.41%)


In [62]:
# Load metadata.yaml from a track folder
def load_metadata(track_path):
    yaml_path = os.path.join(track_path, "metadata.yaml")
    if not os.path.exists(yaml_path):
        return None
    with open(yaml_path, "r") as f:
        return yaml.safe_load(f)

# Search stems based on either program_num or plugin_name
def find_matching_stems(metadata, program_num=None, plugin_name=None):
    if not metadata or "stems" not in metadata:
        return []
    return [
        (stem_id, stem_data)
        for stem_id, stem_data in metadata["stems"].items()
        if (program_num is not None and stem_data.get("program_num") == program_num) or
           (plugin_name is not None and stem_data.get("plugin_name") == plugin_name)
    ]

# Open a file or folder in the system's default file explorer
def open_file_explorer(path):
    if platform.system() == "Windows":
        os.startfile(path)
    elif platform.system() == "Darwin":  # macOS
        subprocess.run(["open", path])
    elif platform.system() == "Linux":
        subprocess.run(["xdg-open", path])
    else:
        print(f"Unsupported operating system: {platform.system()}")

# Find a random track and search for matching stems
def find_random_track(program_num=None, plugin_name=None):
    if program_num is None and plugin_name is None:
        print("You must specify either program_num or plugin_name.")
        return
    
    track_folders = [f for f in os.listdir(DIR_PATH) if f.startswith("Track")]
    if not track_folders:
        print("No tracks found.")
        return
    
    random.shuffle(track_folders)  # Shuffle to pick a random track
    for track in track_folders:
        track_path = os.path.join(DIR_PATH, track)
        metadata = load_metadata(track_path)
        matches = find_matching_stems(metadata, program_num, plugin_name)
        
        if matches:
            print(f"Track: {track}")
            for stem_id, stem_data in matches:
                print(f"  Stem: {stem_id}")
                print(f"    program_num: {stem_data.get('program_num', 'N/A')}")
                print(f"    plugin_name: {stem_data.get('plugin_name', 'N/A')}")
            
            # Open the track folder in the file explorer
            open_file_explorer(track_path)
            return
    
    print("No matching stems found.")

# find_random_track(program_num=6)
find_random_track(plugin_name="trumpet_2.nkm")

Track: Track02024
  Stem: S12
    program_num: 59
    plugin_name: trumpet_2.nkm
